# MongoDB

---

**Источники:**

[Большой туториал по MongoDB](https://medium.com/@Merrick_krg/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D1%82%D1%83%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB-%D0%BF%D0%BE-mongodb-7e2f9e17f0c7)

[MongoDB vs MySQL: A Comparative Study on Databases](https://www.simform.com/mongodb-vs-mysql-databases)

[Academy 3T](https://studio3t.com/academy/)

---

**MongoDB** — документоориентированная система управления базами данных, не требующая описания схемы таблиц.

Считается одним из классических примеров NoSQL-систем. 

Использует JSON-подобные документы и схему базы данных.

Написана на языке C++. 

<center><img src="images/MongoDB_database_colection.png"/></center>

<center><img src="images/rdbms_vs_mongodb.png"/></center>

<center><img src="images/mongodb_and_rdbms_schema.png"/></center>

**MongoDB подходит для следующих применений**:
- хранение и регистрация событий;
- системы управления документами и контентом;
- электронная коммерция;
- игры;
- данные мониторинга, датчиков;
- мобильные приложения;
- хранилище операционных данных веб-страниц (например, хранение комментариев, рейтингов, профилей пользователей, сеансы пользователей).